In [141]:
import pandas as pd
from math import sqrt, log
import sys
import time
import json
import numpy as np
from random import *


class SWGA: 
    
    def __init__(self, bg, fgs, ex, kmer_size=14):
        self.bg = bg
        self.fg = fgs
        self.ex = ex
        self.kmer_size = kmer_size
        self.primer_dict = self.kmers(self.fg, self.bg, self.ex)
        self.binned_data = None
        self.set_dict = None
        
    def _is_sym(self, seq):
        """Returns True if s is symmetric (same as rev. complement)"""
        comp = {
            'A': 'T',
            'T': 'A',
            'G': 'C',
            'C': 'G'
        }
        return seq == ''.join([comp[i] for i in seq][::-1])


    def _overcount(self, st, p):
        ocu = 0
        x = 0
        while True:
            try:
                i = st.index(p, x)
            except ValueError:
                break
            ocu += 1
            x = i + 1
        return ocu


    def _tercorr(self, st):
        _dh = 0
        _ds = -1.4 if self._is_sym(st) else 0
        start = st[0]
        end = st[-1]

        if start == 'G' or start == 'C':
            _dh += 0.1
            _ds -= 2.8
        elif start == 'A' or start == 'T':
            _dh += 2.3
            _ds += 4.1

        if end == 'G' or end == 'C':
            _dh += 0.1
            _ds -= 2.8
        elif end == 'A' or end == 'T':
            _dh += 2.3
            _ds += 4.1
        return _dh, _ds

    def temp(self, s, DNA_c=5000.0, Na_c=10.0, Mg_c=20.0, dNTPs_c=10.0, uncorrected=False):
        '''
        Returns the DNA/DNA melting temp using nearest-neighbor thermodynamics.

        This function returns better results than EMBOSS DAN because it uses updated
        thermodynamics values and takes into account initialization parameters from
        the work of SantaLucia (1998).

        Corrects for mono- and divalent cation concentrations.

        Arguments:
        - DNA_c:   DNA concentration [nM]
        - Na_c:    Na+ concentration [mM]
        - Mg_c:    Mg2+ concentration [mM]
        - dNTPs_c: dNTP concentration [mM]
        - correction: correct for cation concentration?
        '''

        R = 1.987    # Universal gas constant (cal/(K*mol))
        s = s.upper()
        dh, ds = self._tercorr(s)
        k = DNA_c * 1e-9

        # Adapted from Table 1 in Allawi and SantaLucia (1997).
        # delta H (kcal/mol)
        dh_coeffs = {"AA": -7.9, "TT": -7.9,
                     "AT": -7.2,
                     "TA": -7.2,
                     "CA": -8.5, "TG": -8.5,
                     "GT": -8.4, "AC": -8.4,
                     "CT": -7.8, "AG": -7.8,
                     "GA": -8.2, "TC": -8.2,
                     "CG": -10.6,
                     "GC": -9.8,
                     "GG": -8.0, "CC": -8.0}

        # delta S (eu)
        ds_coeffs = {"AA": -22.2, "TT": -22.2,
                     "AT": -20.4,
                     "TA": -21.3,
                     "CA": -22.7, "TG": -22.7,
                     "GT": -22.4, "AC": -22.4,
                     "CT": -21.0, "AG": -21.0,
                     "GA": -22.2, "TC": -22.2,
                     "CG": -27.2,
                     "GC": -24.4,
                     "GG": -19.9, "CC": -19.9}

        # Multiplies the number of times each nuc pair is in the sequence by the
        # appropriate coefficient, then returns the sum of all the pairs
        dh = dh + \
            sum(self._overcount(s, pair) * coeff for pair, coeff in dh_coeffs.items())
        ds = ds + \
            sum(self._overcount(s, pair) * coeff for pair, coeff in ds_coeffs.items())

        fgc = len([filter(lambda x: x == 'G' or x == 'C', s)]) / float(len(s))

        # Melting temperature
        tm = (1000 * dh) / (ds + (R * log(k)))

        if uncorrected:
            return tm - 273.15

        MNa = Na_c * 1e-3
        MMg = Mg_c * 1e-3
        MdNTPs = dNTPs_c * 1e-3

        # Free magnesium concentration
        Ka = 3e4  # association constant in biological buffers
        D = (Ka * MdNTPs - Ka * MMg + 1)**2 + (4 * Ka * MMg)
        Fmg = (-(Ka * MdNTPs - Ka * MMg + 1) + sqrt(D)) / (2 * Ka)

        cation_ratio = sqrt(Fmg) / MNa if MNa > 0 else 7.0

        if cation_ratio < 0.22:
            tm = 1 / (
                (1 / tm) +
                ((4.29 * fgc - 3.95) * log(MNa) + 0.94 * log(MNa)**2) * 1e-5)
        else:
            a = 3.92
            d = 1.42
            g = 8.31
            Fmg = MMg
            if cation_ratio < 6.0:
                a = a * (0.843 - 0.352 * sqrt(MNa) * log(MNa))
                d = d * \
                    (1.279 - 4.03 * log(MNa) * 1e-3 - 8.03 * log(MNa)**2 * 1e-3)
                g = g * (0.486 - 0.258 * log(MNa) + 5.25 * log(MNa)**3 * 1e-3)
            tm = 1 / (
                (1 / tm) +
                (a - 0.911 * log(Fmg) + fgc * (6.26 + d * log(Fmg)) +
                 1 / (2 * (len(s) - 1)) * (-48.2 + 52.5 * log(Fmg) +
                                           g * log(Fmg)**2)) * 1e-5)

        return tm - 273.15

    # Algorithm for actually scoring the primer sets
    def score_kmer(self, primer_dict, c1=0.1, c2=-5, c3=3, default_score=1):
        # FgOccur, BgOccur, degDiff, inEx
        # C1T + C2E + C3*R
        E = 0
        R = primer_dict['FgOccur'] / primer_dict['BgOccur']
        if(primer_dict['InEx'] == True):
            E = -2
        score = c1*default_score + c2*E + c3*R
        return score
    
    # Returns a dict with all foreground occurences
    def get_fg_dict(self, foreground, kmers, min_temp, max_temp):
        with open(foreground, 'r') as fg_file:
            value_dict = {}
            fg_first_line = fg_file.readline()
            fg_data = fg_file.read().replace('\n', '')

            for i in range(int(len(fg_data)) - kmers):
                cur_mer = fg_data[i:i+kmers]
                cur_temp = self.temp(cur_mer)
                if(min_temp <= cur_temp <= max_temp):
                    if(cur_mer in value_dict):
                        value_dict[cur_mer]['FgOccur'] += 1
                        value_dict[cur_mer]['FgLocations'].append(i)
                    else:
                        value_dict[cur_mer] = {'FgOccur': 1, 'FgLocations': [i]}

            return value_dict
    
    # Returns the dict that will be used to write files and store all metadata
    def kmers(self, foregrounds, background, exclusion_set, kmers=14, pref_min=10, pref_max_temp=120, min_temp=0, max_temp=150):

        value_dict = {}
        foreground_dict = {}
        fg_bg_dict = {}
        num_seq = 0
        
        # Loops through all files in the file list you pass
        for foreground in foregrounds:
            
            # Runs get_fg_dict on every file to get all occurences
            print("Forming Kmers: {}".format(foreground))
            fg_dict = self.get_fg_dict(foreground, kmers, min_temp, max_temp)
            foreground_dict[foreground] = fg_dict
        
        # Open the background file and skip the first line
        with open(background, 'r') as bg_file:
            bg_first_line = bg_file.readline()
            bg_data = bg_file.read().replace('\n', '')
            
            # Opens the exclusion file and skips the first line
            with open(exclusion_set, 'r') as ex_file:
                bg_first_line = ex_file.readline()
                ex_data = ex_file.read().replace('\n', '')
                for foreground in foregrounds:
                    value_dict = {}
                    
                    # Iterate through the background file and only store kmer locations that 
                    # are also present in the foreground
                    for i in range(int(len(bg_data)) - kmers):
                        cur_mer = bg_data[i:i+kmers]
                        inEx = False
                        defDiff = 0
                        if(cur_mer in foreground_dict[foreground]):
                            cur_temp = self.temp(cur_mer)
                            if(pref_min > cur_temp):
                                defDiff = pref_min - cur_temp
                            if(pref_max_temp < cur_temp):
                                defDiff = pref_max_temp - cur_temp
                            if(cur_mer in ex_data):
                                inEx = True
                            if(cur_mer in value_dict):
                                value_dict[cur_mer]['BgOccur'] += 1
                                if(i not in value_dict[cur_mer]['BgLocations']):
                                    value_dict[cur_mer]['BgLocations'].append(i)
                            else:
                                value_dict[cur_mer] = {'FgOccur': foreground_dict[foreground][cur_mer]['FgOccur'], 'FgLocations': foreground_dict[foreground][cur_mer]['FgLocations'], 'BgOccur': 1, 'BgLocations': [i], 'MeltingTemp': cur_temp, 'DegreeDiff': defDiff, 'InEx': inEx, 'Score': 0}
                            
                            for key in value_dict.keys():
                                current_mer = value_dict[key]
                                current_mer['Score'] = self.score_kmer(current_mer)
                    
                    # Makes a dict of keys so that we can write to json nicely
                    fg_bg_dict['Background'] = background
                    fg_bg_dict['Foreground'] = foreground
                    fg_bg_dict['Kmers'] = value_dict
                    self.value_dict = value_dict
                    fg_bg_dict['#Sequences'] = len(fg_bg_dict['Kmers'].keys())
                    filename = foreground.split('/')[-1].split('.')[0] + '_' + str(kmers) + 'mer.json'
                    
                    # Writes to json file
                    with open(filename, "w") as json_data_file:
                        json.dump(fg_bg_dict, json_data_file, indent=4, sort_keys=True)

        return fg_bg_dict
    
    # The function that ranks the primers in ascending order
    def rank_primer(self, primer):
        return self.value_dict[primer]['Score']
    
    # Bins the primers and stores that as a value in the class
    def select_primer(self, value_dict, bin_size=5, set_size=5):
        selected_set = []
        if(self.binned_data == None):
            
            # Sorts dict into a list
            sorted_list = sorted(value_dict, key=self.rank_primer)
            sorted_primers = np.array(sorted_list)
            
            # Splits the data into bins
            binned_list = np.array_split(sorted_primers, bin_size)
            self.binned_data = binned_list
        
        # Select all primers randomly
        for i in range(0, set_size - 1):
            selected_bin = randint(0, bin_size - 1)
            selected_seq = randint(0, len(self.binned_data) - 1)
            selected_set.append(self.binned_data[selected_bin][selected_seq])
        return selected_set
    
    # Calculates the Rolling Amplification for each set and returns it
    def get_ra_score(self, primer, selected_set, value_dict, kmer_size, primer_len=7000):        
        cur_locs = value_dict[primer]['FgLocations']
        fg_length = primer_len
        bg_length = primer_len
        for single_primer in selected_set:
            if(single_primer != primer):
                for location in value_dict[single_primer]['FgLocations']:
                    for cur_loc in cur_locs:
                        cur_l = cur_loc + primer_len
                        if((location + primer_len) <= cur_l and (location + primer_len) >= cur_loc):
                            fg_length += (location + primer_len) - cur_l
                            
                
                for location in value_dict[single_primer]['BgLocations']:
                    for cur_loc in cur_locs:
                        cur_l = cur_loc + primer_len
                        if((location + primer_len) <= cur_l and (location + primer_len) >= cur_loc):
                            bg_length += (location + primer_len) - cur_l
        return fg_length, bg_length
                             
    # Run the algorithm that was written up, where C1 is the temp score, and C2 is the exclusion penalty
    def get_best_set(self, selected_set, value_dict, C1=0.5, C2=-2):
        temp_sum = 0
        ex_sum = 0
        total_fg = 0
        total_bg = 0
        n = len(selected_set)
        for primer in selected_set:
            fg_len, bg_len = self.get_ra_score(primer, selected_set, value_dict, self.kmer_size)
            total_fg += fg_len
            total_bg += bg_len
            temp_sum += value_dict[primer]['MeltingTemp']
            if(value_dict[primer]['InEx'] == True):
                ex_sum += 1
        total_ra = total_fg / total_bg
        return selected_set, C1*(temp_sum / n) + C2*(ex_sum) + total_ra
    
    # Function to rank the sets based on their score
    def rank_sets(self, primer):
        return self.set_dict[primer]['Set'][1]
    
    # Runs the set 'num_sets' times and gets the top values (default is 10)
    def get_final_best_set(self, num_sets=1000, top_values=10):
        set_dict = {}
        best_ranked_sets = []
        for iteration in range(0, num_sets):
            primers = AMP.select_primer(AMP.value_dict)
            current_set, current_score = AMP.get_best_set(primers, AMP.value_dict)
            set_dict[iteration] = {'Set' : [current_set, current_score]}
        
        self.set_dict = set_dict
        
        sorted_list = sorted(self.set_dict, key=self.rank_sets)
        
        for i in range(0, top_values):
            best_ranked_sets.append(self.set_dict[sorted_list[i]])
        
        
        return best_ranked_sets
        
            
            
                

In [142]:
bg = "/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/GCF_000007465.2_ASM746v2_genomic.fna"
ex = "/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/exclude.fasta"
fgs = ["/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/Chromosone16.fasta", "/Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/Chromosone17.fasta"]
start = time.process_time()
AMP = SWGA(bg, fgs, ex)
end = time.process_time()
print("Time Taken: {}".format(end - start))

Forming Kmers: /Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/Chromosone16.fasta
Forming Kmers: /Users/thatcher/Documents/Graduate School/FofanovResearch/INF685/data/Chromosone17.fasta
Time Taken: 6.921492000000001


In [149]:
# print(AMP.value_dict)
final_set = AMP.get_final_best_set()
print(len(final_set))
for primer_set in final_set:
    print(primer_set, '\n')
    for primer_val in primer_set['Set'][0]:
        print("Primer: {} \n Metadata: {}".format(primer_val, AMP.value_dict[primer_val]))

10
{'Set': [['AAAAAATAAAATAA', 'CAATATTTTTAAAT', 'CAATATTTTTAAAT', 'CAATATTTTTAAAT'], 22.693782362264102]} 

Primer: AAAAAATAAAATAA 
 Metadata: {'FgOccur': 1, 'FgLocations': [12186], 'BgOccur': 3, 'BgLocations': [211417, 505597, 583045], 'MeltingTemp': 42.621707771957574, 'DegreeDiff': 0, 'InEx': False, 'Score': 1.1}
Primer: CAATATTTTTAAAT 
 Metadata: {'FgOccur': 1, 'FgLocations': [55368], 'BgOccur': 1, 'BgLocations': [1149097], 'MeltingTemp': 43.64285037538508, 'DegreeDiff': 0, 'InEx': False, 'Score': 3.1}
Primer: CAATATTTTTAAAT 
 Metadata: {'FgOccur': 1, 'FgLocations': [55368], 'BgOccur': 1, 'BgLocations': [1149097], 'MeltingTemp': 43.64285037538508, 'DegreeDiff': 0, 'InEx': False, 'Score': 3.1}
Primer: CAATATTTTTAAAT 
 Metadata: {'FgOccur': 1, 'FgLocations': [55368], 'BgOccur': 1, 'BgLocations': [1149097], 'MeltingTemp': 43.64285037538508, 'DegreeDiff': 0, 'InEx': False, 'Score': 3.1}
{'Set': [['AAAAAATAAAATAA', 'CAATATTTTTAAAT', 'ATAAAATAAAAAAA', 'GTCTTATGATAATA'], 22.7241943731054